# Importing Libraries and Data

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import spacy
nlp = spacy.load("en_core_web_sm")



import nltk
nltk.download('stopwords')
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!pip install textdistance --quiet
import textdistance

## Importing Data

In [5]:
url = 'https://raw.githubusercontent.com/ZHarshal4/DataNeuron/main/DataNeuron_Text_Similarity.csv'
documents_df_original = pd.read_csv(url)
documents_df_original.head()

,text1,text2
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...


#Text Processor and Cosine Similarity fn

In [6]:
def text_processing(sentence):

    # Lemmatize, lowercase, remove numbers and stop words, Input sentence , Output list of words

    sentence = [token.lemma_.lower()
                for token in nlp(sentence)
                if token.is_alpha and not token.is_stop]

    return sentence


def cos_sim(sentence1_emb, sentence2_emb):

    # Cosine similarity between two columns of sentence embeddings, Input 2 sentence embeddings like (a,b,c) and (1,2,3)
    # Returns cosine similarity between 2 columns, i.e. similarity between (a,1), (b,2), (c,3)

    cos_sim = cosine_similarity(sentence1_emb, sentence2_emb)
    return np.diag(cos_sim)

#Working with various algorithms

##Jaccard similarity(Non Contextual)

In [ ]:
df_jaccard = documents_df_original.copy()

from tqdm import tqdm
tqdm.pandas()

def jaccard_similarity(row):
    # Text Processing
    sentence1 = text_processing(row['text1'])
    sentence2 = text_processing(row['text2'])

    # Jaccard similarity
    return textdistance.jaccard.normalized_similarity(sentence1, sentence2)


# Jaccard Similarity
df_jaccard['Jaccard_score'] = df_jaccard.progress_apply(jaccard_similarity, axis=1)
df_jaccard

100%|██████████| 3000/3000 [07:48<00:00,  6.40it/s]


,text1,text2,Jaccard_score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.045455
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.022670
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.036145
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.030457
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.047264
...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,0.016611
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,0.034632
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,0.013575
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,0.061350


##Edit Distance(Levenshtein Distance)(Non Contextual)
_Extremely Slow_

In [ ]:
df_edit = documents_df_original.head(20).copy()

def levenshtein_distance(sentence1, sentence2):
    m, n = len(sentence1), len(sentence2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(m + 1):
        dp[i][0] = i
    for j in range(n + 1):
        dp[0][j] = j
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if sentence1[i - 1] == sentence2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i - 1][j - 1], dp[i][j - 1], dp[i - 1][j])
    return dp[m][n]


# Levenshtein Distance
df_edit['levenshtein_distance'] = df_edit.progress_apply(lambda row: levenshtein_distance(row['text1'],row['text2']), axis=1)
df_edit

100%|██████████| 20/20 [01:31<00:00,  4.57s/it]


,text1,text2,levenshtein_distance
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,2104
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,2148
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,1032
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,2251
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,1788
5,india to deport bollywood actress india has or...,fosters buys stake in winemaker australian bre...,1913
6,mutant book wins guardian prize a book about t...,jp morgan admits us slavery links thousands of...,1220
7,aids and climate top davos agenda climate chan...,howl helps boost japan s cinemas japan s box o...,2045
8,kennedy predicts bigger turnout voters pent ...,ocean s twelve raids box office ocean s twelve...,1201
9,carry on star patsy rowlands dies actress pats...,uk broadband gets speed injection broadband s ...,1658


In [ ]:
levenshtein_distance(df_edit.text1[1],df_edit.text2[1])

2148

##TF-IDF
_Considered better than Count Vectorizer as CV treats all words equally important_

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

df_tfidf = documents_df_original.copy()

model = TfidfVectorizer(lowercase=True, stop_words='english')

trainingdata = pd.concat([df_tfidf['text1'], df_tfidf['text2']]).unique()
model.fit(trainingdata)

# Generate Embeddings on Test
sentence1_emb = model.transform(df_tfidf['text1'])
sentence2_emb = model.transform(df_tfidf['text2'])

# Cosine Similarity
df_tfidf['TFIDF_cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)
df_tfidf

,text1,text2,TFIDF_cosine_score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.058171
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.006190
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.006456
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.013339
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.018365
...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,0.002250
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,0.017754
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,0.004470
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,0.012517


##Cosine Distance(with GloVe)

In [7]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2023-12-04 14:33:47--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-12-04 14:33:47--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-12-04 14:33:48--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [8]:
import numpy as np
from scipy.spatial.distance import cosine

df_cosglove = documents_df_original.copy()

# Load pre-trained GloVe word embeddings
embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

similarity = []

for i in range(len(df_cosglove)):
    # Compute sentence embeddings

    words1 = df_cosglove.text1[i].split()
    words2 = df_cosglove.text2[i].split()

    embeddings1 = [embeddings_index.get(word, np.zeros(100)) for word in words1]
    embeddings2 = [embeddings_index.get(word, np.zeros(100)) for word in words2]

    sentence_embedding1 = np.mean(embeddings1, axis=0)
    sentence_embedding2 = np.mean(embeddings2, axis=0)

    # Compute cosine similarity

    similarity.append(cosine(sentence_embedding1, sentence_embedding2))

df_cosglove['Cosine Similarity'] = similarity
df_cosglove


,text1,text2,Cosine Similarity
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.051350
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.044464
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.030633
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.056745
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.035826
...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,0.075521
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,0.062237
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,0.079278
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,0.037542


##WordMoversDistance(with Word2Vec)

In [9]:
import gensim.downloader as api
model_w2v = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [11]:
!pip install pot --quiet
import ot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.0/790.0 kB 8.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/ot/backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


In [14]:
from gensim.models import KeyedVectors
from scipy.spatial.distance import cdist

df_wmdw2v = documents_df_original.copy()

WMDW2V_Distance = []

for i in range(len(df_wmdw2v)):
    # Compute sentence embeddings
    sentence1 = text_processing(df_wmdw2v.text1[i])
    sentence2 = text_processing(df_wmdw2v.text2[i])

    WMDW2V_Distance.append(model_w2v.wmdistance(sentence1, sentence2,norm = True))

# Distance
df_wmdw2v['WMDW2V Distance'] = WMDW2V_Distance
df_wmdw2v.head()

,text1,text2,WMDW2V Distance
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,1.084173
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,1.155408
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,1.092322
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,1.120290
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,1.051594


##WordMoversDistance(with FastText)

In [15]:
import gensim.downloader as api

df_wmdfast = documents_df_original.copy()

In [16]:
# Load the pre-trained model
model_1 = api.load('fasttext-wiki-news-subwords-300')

[==================================================] 100.0% 958.5/958.4MB downloaded


In [19]:
from tqdm import tqdm
tqdm.pandas()

def word_movers_distance(row):
    # Text Processing
    sentence1 = text_processing(row['text1'])
    sentence2 = text_processing(row['text2'])

    # Negative Word Movers Distance
    return -model_1.wmdistance(sentence1, sentence2,norm = True)


# Negative Word Movers Distance
df_wmdfast['NegWMD_score'] = df_wmdfast.progress_apply(word_movers_distance, axis=1)
df_wmdfast

100%|██████████| 3000/3000 [09:32<00:00,  5.24it/s]


,text1,text2,NegWMD_score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,-0.953708
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,-0.963396
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,-0.931430
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,-0.936868
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,-0.897165
...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,-0.979149
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,-0.953496
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,-1.005240
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,-0.882534


##Universal Sentence Encoder

In [ ]:
df_use = documents_df_original.copy()

import tensorflow as tf
import tensorflow_hub as hub

# Load the pre-trained model
gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    # Control GPU memory usage
    tf.config.experimental.set_memory_growth(gpu, True)

module_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'
model = hub.load(module_url)

# Generate Embeddings
sentence1_emb = model(df_use['text1']).numpy()
sentence2_emb = model(df_use['text2']).numpy()

# Cosine Similarity
df_use['cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)
df_use

,text1,text2,cosine_score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.272668
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.277622
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.169010
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.157467
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.246201
...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,0.263255
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,0.323398
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,0.209719
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,0.287181


##SBERT with Sentence Transformer(CrossEncoder)

In [20]:
!pip install sentence_transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.7 MB/s eta 0:00:00


In [23]:
from sentence_transformers import CrossEncoder

df_sbert = documents_df_original.copy()


# Load the pre-trained model
model = CrossEncoder('cross-encoder/stsb-roberta-base')

sentence_pairs = []
for sentence1, sentence2 in zip(df_sbert['text1'], df_sbert['text2']):
    sentence_pairs.append([sentence1, sentence2])

df_sbert['SBERT CrossEncoder_score'] = model.predict(sentence_pairs, show_progress_bar=True)
df_sbert.head(10)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,text1,text2,SBERT CrossEncoder_score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.551049
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.435223
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.438817
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.475700
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.430568
5,india to deport bollywood actress india has or...,fosters buys stake in winemaker australian bre...,0.495987
6,mutant book wins guardian prize a book about t...,jp morgan admits us slavery links thousands of...,0.493592
7,aids and climate top davos agenda climate chan...,howl helps boost japan s cinemas japan s box o...,0.348624
8,kennedy predicts bigger turnout voters pent ...,ocean s twelve raids box office ocean s twelve...,0.463746
9,carry on star patsy rowlands dies actress pats...,uk broadband gets speed injection broadband s ...,0.517548


##SBERT Bi-Encoder with Sentence Transformer

In [24]:
df_sbertbi = documents_df_original.copy()

from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model = SentenceTransformer('stsb-mpnet-base-v2')

# Generate Embeddings
sentence1_emb = model.encode(df_sbertbi['text1'], show_progress_bar=True)
sentence2_emb = model.encode(df_sbertbi['text2'], show_progress_bar=True)

# Cosine Similarity
df_sbertbi['SBERT BiEncoder_cosine_score'] = cos_sim(sentence1_emb, sentence2_emb)
df_sbertbi

.gitattributes:   0%|          | 0.00/868 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.67k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/588 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

,text1,text2,SBERT BiEncoder_cosine_score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.064932
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.089295
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.097306
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,-0.021138
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.223896
...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,0.175267
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,0.138198
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,-0.107069
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,-0.008424


#Deployed Code

## Using BERT MiniLM


In [ ]:
df_mini = documents_df_original.copy()

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

cossimlist =[]
sentence_pairs = []
for sentence1, sentence2 in zip(df_mini['text1'], df_mini['text2']):
    embedding_1 = model.encode(sentence1, convert_to_tensor=True)
    embedding_2 = model.encode(sentence2, convert_to_tensor=True)
    cossimlist.append(util.pytorch_cos_sim(embedding_1, embedding_2).item())
    #df_mini['Predict'] = model.predict(embedding_1, embedding_2)

df_mini['Mini_cosine_score'] = cossimlist
df_mini


#simscore = float("{:.2f}".format(similarity))


,text1,text2,Mini_cosine_score
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.086190
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.165480
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.077801
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,-0.186152
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.088690
...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,0.080345
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,0.097926
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,-0.114992
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,-0.004490
